# MNL

In [28]:
import gurobipy as gp

In [29]:
import numpy as np
import numpy.random as npr
N = 25 #25 50 75 100
M = 5 #5 10 15 20
vm = 100*npr.rand(M, N)
alpha = 1/M * np.ones(M)
print(vm)

[[54.6109308  87.89964847 98.6692781  27.43971335 43.39721133 18.57141365
  49.50383067 84.36347487 90.38320061 75.21006889 65.1782243  56.45384096
  53.93783107 11.3805907  38.06441205 53.6791218  75.65793339 22.22320754
  54.60998157 64.8724912  21.09152181 97.10189348 35.52809382 65.09875334
  88.64476522]
 [23.33942685  0.96284022 88.49679323 68.42145116  9.63160596 62.36733743
  59.10710012 70.20165421 58.4039859  89.19835827  1.86081777 55.67051382
  67.20884703 64.18296183 16.30478704 31.68315768 11.31711597 31.27073646
  90.09504531 57.29725726  5.47592071 75.01996467 80.96556238 62.43636973
  10.05231287]
 [ 6.89339891 50.85811054 17.57331419 42.18666911 73.89579606 75.06189535
  69.01437853 37.89123832 89.36268967 41.69281961 18.2064349  85.70061133
  46.60972437 56.54461449 28.02555    46.43258845 46.03721975 53.90689233
  74.63310998 93.32295836 72.2040011  74.06308488 17.78814805 40.09278516
  43.82396752]
 [87.65644378 76.07581989 49.6778006  86.09377504 59.14346825 25.18

In [30]:
def IP_mmnl(p):
    global vm, N
    global alpha, M

    #create a new model
    myModel = gp.Model("IP_mmnl")

    # create decision variables and store them in the arrays z, x, y
    z = [0 for i in range(M)]
    x = [[0 for j in range(N)] for i in range(M)]
    y = [0 for j in range(N)]
    for i in range(M):
        newVar = myModel.addVar(vtype = gp.GRB.CONTINUOUS, name = "z_" + str(i))
        z[i] = newVar
        for j in range(N):
            newVarx = myModel.addVar(vtype = gp.GRB.CONTINUOUS, name = "x_" + str(i) + '_' + str(j))
            x[i][j] = newVarx
    for j in range(N):
        newVar = myModel.addVar(vtype = gp.GRB.BINARY, name = "y_" + str(j))
        y[j] = newVar
    myModel.update()

    # set objective function
    objExpr = gp.LinExpr()
    for i in range(M):
        objExpr += alpha[i] * z[i]
    myModel.setObjective(objExpr, gp.GRB.MINIMIZE)

    BM = 1000
    # add constraints
    for i in range(M):
        myConstr = gp.LinExpr()
        for j in range(N):
            myConstr += vm[i][j] * x[i][j]
            myModel.addConstr(lhs = x[i][j], sense = gp.GRB.GREATER_EQUAL, rhs = y[j]*BM, name = "x 1st constraint_" + str(i) + '_' + str(j))
            myModel.addConstr(lhs = x[i][j], sense = gp.GRB.LESS_EQUAL, rhs = -y[j]*BM, name = "x 2nd constraint_" + str(i) + '_' + str(j))
            myModel.addConstr(lhs = x[i][j], sense = gp.GRB.GREATER_EQUAL, rhs = p[j] - z[i] - (1-y[j])*BM, name = "x 3rd constraint_" + str(i) + '_' + str(j))
            myModel.addConstr(lhs = x[i][j], sense = gp.GRB.LESS_EQUAL, rhs = p[j] - z[i] + (1-y[j])*BM, name = "x 4th constraint_" + str(i) + '_' + str(j))
        myModel.addConstr(lhs = z[i], sense = gp.GRB.GREATER_EQUAL, rhs = myConstr, name = "z " + str(i))

    # solve model
    myModel.optimize()

    bestS = []
    profit = float(myModel.objVal)
    for i in range(M):
        if y[j].x == 1:
            bestS.append(i)
    return profit, bestS

In [31]:
def profit_mmnl(p, S):
    global vm
    global alpha, M
    pi = 0
    for i in range(M):
        Vi_S = 0
        num = 0
        for j in S:
            num+=p[j]*vm[i][j]
            Vi_S+=vm[i][j]
        pi += alpha[i]*num/(1+Vi_S)
    return pi

In [32]:
def nested_by_price_mmnl(p):
    global vm, N
    global alpha, M
    max_profit = -1
    best_S = []
    for k in range(N):
        S = [j for j in range(k+1)]
        if profit_mmnl(p, S) >= max_profit:
            best_S = S
            max_profit = profit_mmnl(p, S)
    best_S = sorted(best_S)
    return max_profit, best_S

In [33]:
def find_best_option_mmnl(p, S):
    global vm, N
    current_best = profit_mmnl(p, S)
    bestS = S
    flag = 0
    N_S = [k for k in range(N) if k not in S]
    for k in N_S:
        newS = S + [k]
        if profit_mmnl(p, newS) >= current_best:
            flag = 1
            bestS = newS
            current_best = profit_mmnl(p, newS)
    return current_best, bestS, flag

def greedy_mmnl(p):
    global vm, N
    max_profit = -1
    best_S = []
    S = []
    flag = 1
    while flag==1:
        max_profit, S, flag = find_best_option_mmnl(p, S)
    best_S = sorted(S)
    return max_profit, best_S

In [34]:
import matplotlib.pyplot as plt
import math
import statistics as st
import time

In [35]:
n_simulations = 10000
avg_price = 100
std_price = 50

In [36]:
count_g = 0
result_g = np.zeros(n_simulations)
for i in range(n_simulations):
    p = npr.exponential(avg_price, N)
    # p = npr.normal(avg_price, std_price, N)
    p = sorted(p, reverse = True)
    gr = greedy_mmnl(p)
    mipl = IP_mmnl(p)
    result_g[i] = 100*gr[0]/mipl[0]
    if gr[1] != mipl[1]:
        count_g+=1
print(count_g)
print(min(result_g))
print(st.mean(result_g))
print(max(result_g))

AttributeError: module 'gurobipy' has no attribute 'GREATER_EQUAL'